<a id="top"></a>
# Downloading UV spectra – Part 1
***

## Learning Goals

By the end of this tutorial, you will be able to:

- Access data from the MAST archives directly from Python
- Perform data queries by Target Name and narrow them down to the desired spectrum
- Save arrays onto _.txt_ files
- Implement Python functions

## Introduction

**Extinction curve background**: An extinction curve represents the wavelength dependence of dust extinction. It compares the inherent Spectral Energy Distribution (SED) (~dust-free) of a star to the observed SED affected by dust extinction. Normally these curves are created by plotting $k(\lambda-V)$ versus $1/\lambda$, with $\lambda$ being the wavelength.

Extinction is relevant in many different scenarios. Dust can be found very near the observed object – such as stars having disks or proto-stellar clouds surrounding it – or can be far away from it but still affect the observation performed – for example dust in a galaxy that happens to be between the observer and the star.

**International Ultraviolet Explorer (IUE)**: The IUE performed spectrophotometry in the wavelength range going from 1150 Å to 3200 Å. More than 100,000 spectra collected by the IUE between 1978 and 1996 can be accessed via the [MAST portal](https://archive.stsci.edu/iue/).

**Magellanic Clouds**: irregular dwarf galaxies that orbit the Milky Way galaxy and are located in the southern celestial hemisphere. Two distinct groups can be differentiated: the Large Magellanic Cloud (LMC) and the Small Magellanic Cloud (SMC).

**Defining some terms**:

- **Color index**: difference between magnitude of a star in 2 different passbands, typically B and V. Symbol: $(B-V)$. 
- **Extinction**: absoption and scattering of light by dust and gas between an object and the observer. It is a measure of the interstellar reddening quantified by the difference between the magnitude of the band when observed through dust versus a dust-free environment. Symbol: $A(\lambda)$.  

- **Spectral type**: stellar classification from hotter (O stars) to cooler (M stars). Temperature defines a star's "color" and surface brightness.


**Useful equations**
- $k(\lambda-V) = \frac{m(\lambda-V)-m(\lambda-V)_o}{(B-V)-(B-V)_o} = \frac{A(\lambda)-A(V)}{A(B)-A(V)}$

_**Note**: the $x_o$ terms refer to the star that is nearly unaffected by dust, i.e. $(B-V)$ corresponds to the observed color index and $(B-V)_o$ to the observed color index if there was no extinction due to dust. The stars should have the same spectral type in order to perform this comparison._
    
This tutorial is divided in two parts. In Part 1, you will learn how to navigate the IUE database through the MAST portal, download and save to your local computer the fluxes and their corresponding wavelengths of different stars, one reddened and one unreddened one. In Part 2, we will use these stars spectra to construct and plot UV extinction curves.  
    
## Table of Contents
* [Imports](#Imports)
* [Searching the MAST archive by target name and downloading spectra](#Searching-the-MAST-archive-by-target-name-and-downloading-spectra)
* [Exploring the downloaded data files](#Exploring-the-downloaded-data-files)
* [Data access and manipulation](#Data-access-and-manipulation)
* [Saving values into your local computer](#Saving-values-into-your-local-computer)
* [Exercises](#Exercises)
* [Additional Resources](#Additional-Resources)

## Imports 

The first step will be to import the libraries we will be using throughout this tutorial: 

- `Observations` from _astroquery.mast_ to query the Barbara A. Mikulski Archive for Space Telescopes (MAST).
- `fits` from _astropy.io_ for accessing FITS files
- `numpy` for array manipulations

In [ ]:
from astroquery.mast import Observations
from astropy.io import fits
import numpy as np

_**Warning**: If you have not installed the astroquery package in your computer, you may need to. Information about astroquery can be found <a href="https://astroquery.readthedocs.io/en/latest/">here</a>._

## Searching the MAST archive by target name and downloading spectra

The next step is to find the data file we will use. This is similar to searching through the <a href="https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html">MAST portal</a> since we will be using specific keywords to find the file. We will be using the `Observations` from `astroquery.mast` to download UV data directly from the IUE archive.

The target name of the first star we will be studying is AzV 214 (observation ID: swp22372), collected by International Ultraviolet Explorer (IUE). This star is known to be reddened by dust. We are directly taking this example from [Gordon et al. (2003)](https://ui.adsabs.harvard.edu/abs/2003ApJ...594..279G/abstract), but any pair of reddened and unreddened stars which have the same spectral type and luminosity class may be used. We will be using Gordon et al. (2003) because we want to explore the differences in extinction curves fro stars belonging to the LMC and the SMC, which will become clear by the end of Part 2. 

To get the data products we want we need the observation ID corresponding to it, in order to do this we can first query the collection of IUE using _Observations_ by using the target name as shown below.

In [ ]:
IUEobs_av214 = Observations.query_criteria(objectname="AzV214",obs_collection="IUE")
data_products_av214 = Observations.get_product_list(IUEobs_av214)

data_products_av214

As we can see from the obtained table, there are many different observations available for our target in the database. We specifically want to focus on sciences products, to know whether an observation is one we need to look at the _productType_ column, science products have the label 'SCIENCE'. Additionally, we will be using extracted spectra via the [Virtual Observatory (VO)](https://ivoa.net) using the [Simple Spectral Access Protocol (SSAP)](https://www.ivoa.net/documents/SSA/). The SSAP is the recommended way by VO to remotely access one dimensional spectra. In order to ensure this, the _description_ column should contain '(extracted spectra/vo spectral container/SSAP) fits file.'

Two different observations, "swp22372" for the short wavelength domain and "lwr17263" for the long wave domain, have been chosen in order to provide a more complete understanding of the extinction behaviour over different wavelengths. For this tutorial we have chosen these specific observations, but the reader is welcome to use any other of the IDs displayed on the table above, as long as they fulfill the conditions desribed in the previous paragraph.  It should be taken into account that some of the data may need some prior calibration before using it. 

- Observation ID: "swp22372":

In [ ]:
data_products_av214[(data_products_av214['obs_id'] == 'swp22372') & (data_products_av214['productType'] == 'SCIENCE') & (data_products_av214['description'] == '(extracted spectra/vo spectral container/SSAP) fits file')]

Great! We have found the product that we want to download. By using this information we can download it by using its observation ID. Additionally, the _productFilename_ displayed in the table above will be useful to directly get the product we desire. We can use in `Observations.filter_products` under the parameter _extension_ as shown below:

In [ ]:
IUEobs_av214_sw = Observations.query_criteria(obs_collection="IUE", obs_id="swp22372")
data_products_av214_sw = Observations.get_product_list(IUEobs_av214_sw)
yourProd_av214_sw = Observations.filter_products(data_products_av214_sw, extension='swp22372mxlo_vo.fits')

yourProd_av214_sw

Now that we have the data file, we can download it by using the `download_products` function:

In [ ]:
Observations.download_products(yourProd_av214_sw) 

If the download does not happen automatically, by visiting the URL displayed by the `download_products` method you will be able to download the file. 

As stated before, in order to have a more  complete understanding of the extinction curve across a broad range of wavelengths, let's download a different observation for the same object, but in a different wavelength range. We can use the following observation ID: lwr17263 (this still corresponds to AzV 214).

- Observation ID: "lwr17263":

In [ ]:
data_products_av214[(data_products_av214['obs_id'] == 'lwr17263') & (data_products_av214['productType'] == 'SCIENCE') & (data_products_av214['description'] == '(extracted spectra/vo spectral container/SSAP) fits file')]

In [ ]:
IUEobs_av214_lw = Observations.query_criteria(obs_collection="IUE", obs_id="lwr17263")
data_products_av214_lw = Observations.get_product_list(IUEobs_av214_lw)
yourProd_av214_lw = Observations.filter_products(data_products_av214_lw, extension='lwr17263mxlo_vo.fits')
Observations.download_products(yourProd_av214_lw) 

Now let's do the same for the unreddened star. The target name of this star is AzV 380 (observation IDs chosen: swp10319, lwr17265) and was also collected by IUE.

In [ ]:
IUEobs_av380 = Observations.query_criteria(objectname="AV380",obs_collection="IUE")
data_products_av380 = Observations.get_product_list(IUEobs_av380)

- Observation ID: "swp10319":

In [ ]:
data_products_av380[(data_products_av380['obs_id'] == 'swp10319') & (data_products_av380['productType'] == 'SCIENCE') & (data_products_av380['description'] == '(extracted spectra/vo spectral container/SSAP) fits file')]

In [ ]:
IUEobs_av380_sw = Observations.query_criteria(obs_collection="IUE", obs_id="swp10319")
data_products_av380_sw = Observations.get_product_list(IUEobs_av380_sw)
yourProd_av380_sw = Observations.filter_products(data_products_av380_sw, extension='swp10319mxlo_vo.fits')
Observations.download_products(yourProd_av380_sw) 

- Observation ID: "lwr17265":

In [ ]:
data_products_av380[(data_products_av380['obs_id'] == 'lwr17265') & (data_products_av380['productType'] == 'SCIENCE') & (data_products_av380['description'] == '(extracted spectra/vo spectral container/SSAP) fits file')]

In [ ]:
IUEobs_av380_lw = Observations.query_criteria(obs_collection="IUE", obs_id="lwr17265")
data_products_av380_lw = Observations.get_product_list(IUEobs_av380_lw)
yourProd_av380_lw = Observations.filter_products(data_products_av380_lw, extension='lwr17265mxlo_vo.fits')
Observations.download_products(yourProd_av380_lw) 

Again, if the download does not happen automatically, by visiting the URLs provided by `download_products` you will be able to download the necessary data to continue this tutorial.

## Exploring the downloaded data files

Now, let's explore the FITS file that we got for the reddened star. The paths to the files should correspond to wherever these files are located on your local computer, here the provided path is the one automatically generated by the `download_products` function. Feel free to change the filepath if you have stored the files in a different directory.

In [ ]:
filename_av214_sw = "./mastDownload/IUE/swp22372/swp22372mxlo_vo.fits"
fits.info(filename_av214_sw)

- **No. 0 (PRIMARY)**: This HDU contains meta-data related to the entire file.
- **No. 1 (Spectral Container)**: This HDU contains the spectral profile of the target as a function of wavelength.

The header of the file contains additional information about the data. It can be accessed in the following way (only some part of the information contained in the header is printed in this section, which tells us information about what each column of the data contained in it represents. The user is encouraged to print the complete header if they are interested to see the whole information contained in it).  

In [ ]:
with fits.open(filename_av214_sw) as hdulist: 
    header_av214_sw = hdulist[1].header

print(repr(header_av214_sw[9:15]))

From this we know that the first column corresponds to the wavelengths, and the second column to the fluxes. The header also has information regarding the unit system used:

In [ ]:
print(repr(header_av214_sw[23:28]))

## Data access and manipulation

The data contained in this fits file can be accessed using io.fits and the `data` attribute:

In [ ]:
with fits.open(filename_av214_sw) as hdulist:
    spectrum_av214_sw = hdulist[1].data

wav_av214_sw = spectrum_av214_sw[0][0] # angstrom, A
flux_av214_sw = spectrum_av214_sw[0][1] # ergs/cm2/sec/A

filename_av214_lw = "./mastDownload/IUE/lwr17263/lwr17263mxlo_vo.fits"
with fits.open(filename_av214_lw) as hdulist:
    spectrum_av214_lw = hdulist[1].data

wav_av214_lw = spectrum_av214_lw[0][0] # A
flux_av214_lw = spectrum_av214_lw[0][1] # ergs/cm2/sec/A

_**Heads-up**: some values of the flux may be contaminated or altered, and when applying the logarithm may result in NaN (Not a Number)_

In order to not have to deal with error messages throughout the notebook, we will use the `changeNaN` function displayed below to locate and change these problematic values. The change NaN function below examines a pair of wavelength and flux pairs. Since we will need to combine the reddened and un-reddened stars later on, they need to have the same length, we'll use it on both stars simultaneously. It will delete the rows of all columns (fluxes and wavelengths) if either star's flux value is negative in that row.

An example of the problematic values can be seen in _flux_av214_lw_:

In [ ]:
print("flux_av214_lw:", flux_av214_lw[-5:-3])
print("log(flux_av214_lw):", np.log10(flux_av214_lw[-5:-3]))

In [ ]:
def changeNaN(wavv,fluxx,wavv1,fluxx1):
    
    """
    Removes negative flux values to avoid encountering NaN when converting to flux scale
    
    Inputs:
    :param wavv,wavv1: input wavelength arrays
    :type wavv,wavv1: float array
    :param fluxx,fluxx1: input wavelength arrays
    :type fluxx,fluxx1: float array
    
    Outputs: 
    :return: wavv,fluxx,wavv1,fluxx1 without the corresponding problematic values
    :rtype: float array
    """
    
    prob = np.where(fluxx < 0)[0]
    wavv = np.delete(wavv,prob)
    fluxx = np.delete(fluxx,prob)
    wavv1 = np.delete(wavv1,prob)
    fluxx1 = np.delete(fluxx1,prob)
    
    prob2 = np.where(fluxx1 < 0)[0]
    wavv = np.delete(wavv,prob2)
    fluxx = np.delete(fluxx,prob2)
    wavv1 = np.delete(wavv1,prob2)
    fluxx1 = np.delete(fluxx1,prob2)
    
    return wavv, fluxx, wavv1, fluxx1

Let's do the same for the other files, use the `changeNaN` function to remove the problematic values.

In [ ]:
filename_av380_sw = "./mastDownload/IUE/swp10319/swp10319mxlo_vo.fits"
with fits.open(filename_av380_sw) as hdulist:
    spectrum_av380_sw = hdulist[1].data

wav_av380_sw = spectrum_av380_sw[0][0] # A
flux_av380_sw = spectrum_av380_sw[0][1] # ergs/cm2/sec/A

filename_av380_lw = "./mastDownload/IUE/lwr17265/lwr17265mxlo_vo.fits"
with fits.open(filename_av380_lw) as hdulist:
    spectrum_av380_lw = hdulist[1].data

wav_av380_lw = spectrum_av380_lw[0][0] # A
flux_av380_lw = spectrum_av380_lw[0][1] # ergs/cm2/sec/A

# Remove problematic values
wav_av214_sw, flux_av214_sw, wav_av380_sw, flux_av380_sw = changeNaN(wav_av214_sw, flux_av214_sw, wav_av380_sw, flux_av380_sw)
wav_av214_lw, flux_av214_lw, wav_av380_lw, flux_av380_lw = changeNaN(wav_av214_lw, flux_av214_lw, wav_av380_lw, flux_av380_lw)

In Part 2 of this tutorial we will explore how to visualize the spectra that we have just obtained.

## Saving values into your local computer

Now that we have removed the problematic values from our observations, let's explore how we can save them to our computers. To do this, we can use the function `saveArray` presented below:

In [ ]:
def saveArray(array,filename):
    
    """
    Saves arrays into .txt files in current directory and checks that the files have been written properly
    
    Inputs:
    :param array: array to be saved to local directory
    :type array: float array
    :param filename: name of file to be saved
    :type filename: string
    
    Outputs: 
    :return: none
    :rtype: none
    """
    
    # First display the start and ending of the array to check later
    print('First 3 elements of original array:\n', array[:3])
    print('Last 3 elements of original array:\n', array[-3:])

    # Save the array in a text file
    np.savetxt(filename,array)
    
    # Display the saved content to check it was correctly written to the .txt file
    content = np.loadtxt(filename)
    print('First 3 elements of '+filename+':\n', content[:3])
    print('Last 3 elements of '+filename+':\n', content[-3:],'\n')
    
    return None

In [ ]:
arrays = [wav_av214_sw, flux_av214_sw, wav_av380_sw, flux_av380_sw, wav_av214_lw, flux_av214_lw, wav_av380_lw, flux_av380_lw]
filenames = ['wav_av214_sw.txt', 'flux_av214_sw.txt', 'wav_av380_sw.txt', 'flux_av380_sw.txt', 'wav_av214_lw.txt', 'flux_av214_lw.txt', 'wav_av380_lw.txt', 'flux_av380_lw.txt']

for i in range(len(arrays)):
    saveArray(arrays[i],filenames[i])

## Exercises

Now you can try to do it yourself! Try to obtain the spectra necessary to create an extinction curve of a star belonging to the Large Magellanic Cloud (LMC) following the steps presented for the SMC one.

### 1. Load the data

The targets for this exercise will be:

- Sk -69 206 (Observation IDs: swp36552, lwp15751) (reddened star)
- Sk -67 5 (Observation IDs: swp04827, lwr04170) (unreddened star)

First, do the database search for the reddened star:

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


Narrow down the possibilities to the desired observation, observation ID: "swp36552":

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


Now, download the data as before:

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


And again, in order to cover a wider wavelength range, let's also download the other part of the spectra using the second observation ID provided, observation ID: "lwp15751":

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


Now, let's do the same for the unreddened star:

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


- Observation ID: "swp04827":

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


- Observation ID: "lwr04170"

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


### 2. Open the fits file and explore it

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


### 3. Save the data to your local computer

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


## Additional Resources

For more information about the MAST archive and details about mission data: 

* <a href="https://mast.stsci.edu/api/v0/index.html">MAST API</a> <br>
* <a href="https://archive.stsci.edu/iue/">International Ultraviolet Explorer Page (MAST)</a> <br>
* <a href="https://archive.stsci.edu/iue/mdr_help.html">IUE Data Retrieval Help Page</a> 

For more information about extinction curves and their parametrization:


* <a href="https://ui.adsabs.harvard.edu/abs/2003ApJ...594..279G/abstract">A quantitative comparison of SMC, LMC and Milky Way UV to NIR extinction curves</a> <br>
* <a href="https://adsabs.harvard.edu/pdf/1990ApJS...72..163F">An analysis of the shapes of the ultraviolet extinction curves. III. An atlas of ultraviolet extinction curves</a> 

## About this Notebook

**Author**: Clara Puerto Sánchez <br>
**Keyword(s)**: Tutorial, UV, reddening, extinction-curve <br>
**Last Updated**: Sep 2022 <br>
**Next review**: Apr 2023

For support, please contact the Archive HelpDesk at archive@stsci.edu.
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>